In [77]:
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import time

In [78]:
class MRM(object):
    """ Class for a single microring modulator. """

    Vd = None
    Id = None
    
    LowLim = 0
    HiLim = 4
    
    def __init__(self, SMU):

        self.SMU = SMU

        # Connect instruments
        self.SMU.connect()
        self.SMU.output_off()

    def apply_voltage(self, V):
        """Apply bias to heater."""    
        self.SMU.source_voltage(V)

    def measure_current(self):
        """Measure the photocurrent generated by the microring resonator using the source meter unit (SMU)."""
        return self.SMU.measure_current()

    def update(self, V):
        """Apply the command (heater power) and measure the output (photocurrent)."""
        self.apply_voltage(V)
        return self.measure_current()[0]
    
    def sweep(self,V):
        """Sweep the voltage and measure the photocurrent."""
        I = []
        for v in V:
            I.append(self.update(v))
        plt.plot(V,np.absolute(np.asarray(I)*1e3), label = self.get_variable_name()[0])
        plt.xlabel("Voltage [V]")
        plt.ylabel("Current [mA]")
        plt.legend()
        plt.show()
        return V,I
        
    def calibrate(self):
        user_input = raw_input("Calibration requires that no optical power is present on the chip. Proceed? (y/n)\n")
        if user_input == "y":
            print("Calibration in progress ...\n")
            self.SMU.output_on()
            self.Vd, self.Id = self.sweep(self.V)
            self.SMU.output_off()
            print("Calibration finished!\n")
        else:
            print("Calibration aborted.\n")
            
    
    def update_norm(self, V, delay = 0.5):

        # Apply voltage
        self.apply_voltage(V)
        
        time.sleep(delay)
        
        # Measure the current
        I = self.measure_current()[0]
        
        # Remove the resistive component
        V_closest = min(self.Vd, key=lambda x:abs(x-V))
        In = I - self.Id[self.Vd.index(V_closest)]
        
        return In
        
    def measure_Id(self, index):

        self.apply_voltage(self.V[index])
        
        V_closest = min(self.Vd, key=lambda x:abs(x-self.V[index]))
        Id = self.measure_current()[0] - self.Id[self.Vd.index(V_closest)]
        return Id
        
    def sweep_norm(self):
        """Sweep the voltage and measure the photocurrent."""
        raw_input("Turn on the laser pefore proceeding! Press any key to continue.")
        self.SMU.output_on()
        I = []
        for v in self.V:
            I.append(self.update_norm(v, 0))
        plt.plot(range(len(self.V)),np.asarray(I)*1e6, label = self.get_variable_name()[0])
        plt.xlabel("Voltage [V]")
        plt.ylabel("Current [uA]")
        plt.legend()
        plt.show()
        self.SMU.output_off()
        #return V,I
        
    def get_variable_name(self):
        return [k for k,v in globals().items() if v is self]

In [79]:
import time
def IQ_FBL(MRM_I,MRM_Q, I_iI, I_iQ, d_indexI, d_indexQ, N_iter, I_on = True, Q_on = True):
    """Feedback look for resonance alignment."""
    
    # Configure options
    indexI = I_iI
    indexQ = I_iQ
    Iter = range(N_iter)
    Iteration, V_inI, I_outI, V_inQ, I_outQ = [], [], [], [], []
    
    # Configure plot I
    fig, (ax1, ax3) = plt.subplots(1, 2)
    ax1.set_title('I Tuning')
    ax3.set_title('Q Tuning')
    
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Voltage [V]', color='b')
    ax1.tick_params('y', colors='b')
    ax2 = ax1.twinx()
    ax2.set_ylabel('Photocurrent [uA]', color='r')
    ax2.tick_params('y', colors='r')
    
    # Configure plot Q
    ax3.set_xlabel('Iteration')
    ax3.set_ylabel('Voltage [V]', color='b')
    ax3.tick_params('y', colors='b')
    ax4 = ax3.twinx()
    ax4.set_ylabel('Photocurrent [uA]', color='r')
    ax4.tick_params('y', colors='r')
    
    # Feedback loop
    I.SMU.output_on()
    Q.SMU.output_on()
    for i in Iter:
    
       # time.sleep(0.2)
    
        X_I = MRM_I.measure_Id(indexI)     # Measure X
        X_Q = MRM_Q.measure_Id(indexQ)     # Measure X
    
        if I_on:    
            indexI += d_indexI              # Increase the voltage
        if Q_on:
            indexQ += d_indexQ              # Increase the voltage

        Y_I = MRM_I.measure_Id(indexI)     # Measure Y
        Y_Q = MRM_Q.measure_Id(indexQ)     # Measure Y

        if Y_I>X_I:
            d_indexI = -1 * d_indexI
            
        if Y_Q>X_Q:
            d_indexQ = -1 * d_indexQ
    
        # Actuate values
        Iteration.append(i)
        V_inI.append(MRM_I.V[indexI])
        I_outI.append(Y_I*1e6)
        V_inQ.append(MRM_Q.V[indexQ])
        I_outQ.append(Y_Q*1e6)

        # Draw the plot
        ax1.plot(Iteration, V_inI, 'b')
        ax2.plot(Iteration, I_outI, 'r')
        ax1.set_xlim([min(Iteration),max(Iteration)])
        if len(Iteration)>100:
            ax1.set_xlim([max(Iteration)-100,max(Iteration)])
        ax1.set_ylim([min(V_inI),max(V_inI)])
        ax2.set_ylim([min(I_outI),max(I_outI)])
        
        ax3.plot(Iteration, V_inQ, 'b')
        ax4.plot(Iteration, I_outQ, 'r')
        ax3.set_xlim([min(Iteration),max(Iteration)])
        if len(Iteration)>100:
            ax3.set_xlim([max(Iteration)-100,max(Iteration)])
        ax3.set_ylim([min(V_inQ),max(V_inQ)])
        ax4.set_ylim([min(I_outQ),max(I_outQ)])
        plt.pause(0.01)
        
        #Average
        ax1.cla()
        ax2.cla()
        ax3.cla()
        ax4.cla()
        ax1.plot([min(Iteration),max(Iteration)+1], [np.average(np.asarray(V_inI)),np.average(np.asarray(V_inI))], 'b--')
        ax2.plot([min(Iteration),max(Iteration)+1], [np.average(np.asarray(I_outI)),np.average(np.asarray(I_outI))], 'r--')
        ax3.plot([min(Iteration),max(Iteration)+1], [np.average(np.asarray(V_inQ)),np.average(np.asarray(V_inQ))], 'b--')
        ax4.plot([min(Iteration),max(Iteration)+1], [np.average(np.asarray(I_outQ)),np.average(np.asarray(I_outQ))], 'r--')
        
    fig.tight_layout()
    ax1.set_xlim([min(Iteration),max(Iteration)])
    ax3.set_xlim([min(Iteration),max(Iteration)])
    fig.show()
    
    # Save the data
    timestr = time.strftime("%Y-%m-%d@%H_%M_%S")
    np.savetxt('IQ_' + timestr + '.txt', (Iteration, V_inI, I_outI, V_inQ, I_outQ))

In [86]:
# Initialize I
from Instruments.Keithley_2612B import Keithley_2612B
I = MRM(Keithley_2612B(1, 26, 'a'))

# Initialize Q
from Instruments.Keithley_2612B import Keithley_2612B
Q = MRM(Keithley_2612B(1, 26, 'b'))

# Set the voltage range + resolution
I.V = np.linspace(0,4,1000).tolist()
Q.V = np.linspace(0,4,1000).tolist()

# Calibration
I.calibrate()
Q.calibrate()

Calibration requires that no optical power is present on the chip. Proceed? (y/n)
y
Calibration in progress ...

Calibration finished!

Calibration requires that no optical power is present on the chip. Proceed? (y/n)
y
Calibration in progress ...

Calibration finished!



In [87]:
# Find Initial value for power 
I.sweep_norm()
Q.sweep_norm()

Turn on the laser pefore proceeding! Press any key to continue.g
Turn on the laser pefore proceeding! Press any key to continue.f


In [88]:
%matplotlib qt
# Running the feedback loop
IQ_FBL(I,Q, 570,470, 1,1, 4000, True, True)

C:\ProgramData\Anaconda2\lib\site-packages\matplotlib\axes\_base.py:3471: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=2.28628628629, top=2.28628628629
  'bottom=%s, top=%s') % (bottom, top))
C:\ProgramData\Anaconda2\lib\site-packages\matplotlib\axes\_base.py:3471: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-8.43, top=-8.43
  'bottom=%s, top=%s') % (bottom, top))
C:\ProgramData\Anaconda2\lib\site-packages\matplotlib\axes\_base.py:3471: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=1.88588588589, top=1.88588588589
  'bottom=%s, top=%s') % (bottom, top))
C:\ProgramData\Anaconda2\lib\site-packages\matplotlib\axes\_base.py:3471: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.37, top=-0

KeyboardInterrupt: 

In [143]:
Q.SMU.output_off()
Q.SMU.source_voltage(0)